In [4]:
import pandas as pd
import numpy as np
import pypsa
import yaml

In [44]:
with open('../config.yaml', 'r') as configfile:
    config = yaml.safe_load(configfile)

In [45]:
### config from config.yaml

TY = config['scenario']['target_years'][0]

simulation_year = config['scenario']['simulation_years'][0]

short_names = config['carriers_short_names']

basedir = f'/srv/ES2050/ensym/DestinE/ERAA/scenario_{TY}_{simulation_year}/'

scenario = f'National estimates {TY}' # snakemake input!

network_f = f'../networks/pilot_elec_TY{TY}_{simulation_year}.nc'  # snakemake input!

In [46]:
network = pypsa.Network(network_f)
network.name = 'DestinE Pilot with VRE profiles'

Index(['UA00 nuclear', 'UA00 coal', 'UA00 ror', 'UA00 onwind', 'UA00 CSP',
       'UA00 solar', 'UA00 other RE'],
      dtype='object', name='name')
Index(['UA01 - Demand', 'UA00 - Demand'], dtype='object', name='name')
INFO:pypsa.io:Imported network pilot_elec_TY2030_2010.nc has buses, carriers, generators, links, loads, stores


In [47]:
vre_p_max_pu = pd.read_csv(basedir+f'generation_vre_{simulation_year}.csv', header=0, index_col=0)

vre_gens = pd.DataFrame(index=vre_p_max_pu.columns)
vre_gens['carrier'] = [c.split(' - ')[1] for c in vre_p_max_pu.columns]
vre_gens['carrier_short'] = vre_gens['carrier'].map(short_names)
vre_gens['node'] = [c.split(' - ')[0] for c in vre_p_max_pu.columns]
vre_gens['name_short'] = vre_gens[['node', 'carrier_short']].agg(' '.join, axis=1)

vre_p_max_pu.columns = vre_p_max_pu.columns.map(vre_gens['name_short'])
vre_p_max_pu.index = network.snapshots

In [55]:
### add inflow to run of river plants
inflow = pd.read_csv(basedir+f'hydro_inflows_{simulation_year}.csv', header=0, index_col=0)

hydro_gens = pd.DataFrame(index=inflow.columns)
hydro_gens['carrier'] = [c.split(' - ')[1] for c in inflow.columns]
hydro_gens['carrier_short'] = hydro_gens['carrier'].map(short_names)
hydro_gens['node'] = [c.split(' - ')[0] for c in inflow.columns]
hydro_gens['name_short'] = hydro_gens[['node', 'carrier_short']].agg(' '.join, axis=1)

inflow.columns = inflow.columns.map(hydro_gens['name_short'])
inflow.index = network.snapshots

inflow = inflow.loc[:, inflow.columns[inflow.columns.isin(network.generators.index)]]
inflow_pu = (inflow / network.generators.p_nom[inflow.columns]).clip(upper=1.)

In [56]:
p_max_pu = pd.concat([vre_p_max_pu, inflow_pu], axis=1)
network.generators_t.p_max_pu = p_max_pu[p_max_pu.columns[p_max_pu.columns.isin(network.generators.index)]]

In [15]:
network.export_to_netcdf(f'../networks/pilot_elec-vre_TY{TY}_{simulation_year}.nc')

INFO:pypsa.io:Exported network pilot_elec-vre_TY2030_2010.nc has loads, carriers, links, storage_units, generators, buses


<xarray.Dataset>
Dimensions:                                (snapshots: 8760,
                                            investment_periods: 0,
                                            loads_i: 184, loads_t_p_set_i: 127,
                                            carriers_i: 18, links_i: 227,
                                            links_t_p_min_pu_i: 12,
                                            links_t_p_max_pu_i: 18,
                                            storage_units_i: 115,
                                            generators_i: 676,
                                            generators_t_p_max_pu_i: 178,
                                            buses_i: 60)
Coordinates:
  * snapshots                              (snapshots) int64 0 1 2 ... 8758 8759
  * investment_periods                     (investment_periods) int64 
  * loads_i                                (loads_i) object 'AL00' ... 'DE00 HP'
  * loads_t_p_set_i                        (loads_t_p_set_i) object 'AL00' .....
  * carriers_i                             (carriers_i) object 'Nuclear/-' .....
  * links_i                                (links_i) object 'AL00-GR00 HVAC' ...
  * links_t_p_min_pu_i                     (links_t_p_min_pu_i) object 'BE00-...
  * links_t_p_max_pu_i                     (links_t_p_max_pu_i) object 'DE00-...
  * storage_units_i                        (storage_units_i) object 'AT00 Hyd...
  * generators_i                           (generators_i) object 'AL00 Nuclea...
  * generators_t_p_max_pu_i                (generators_t_p_max_pu_i) object '...
  * buses_i                                (buses_i) object 'AL00' ... 'UKNI'
Data variables: (12/31)
    snapshots_snapshot                     (snapshots) datetime64[ns] 2010-01...
    snapshots_objective                    (snapshots) float64 1.0 1.0 ... 1.0
    snapshots_generators                   (snapshots) float64 1.0 1.0 ... 1.0
    snapshots_stores                       (snapshots) float64 1.0 1.0 ... 1.0
    investment_periods_objective           (investment_periods) float64 
    investment_periods_years               (investment_periods) float64 
    ...                                     ...
    generators_marginal_cost               (generators_i) float64 9.0 ... 0.0
    generators_p_nom                       (generators_i) float64 0.0 ... 18.0
    generators_carrier                     (generators_i) object 'Nuclear/-' ...
    generators_t_p_max_pu                  (snapshots, generators_t_p_max_pu_i) float64 ...
    buses_x                                (buses_i) float64 19.91 ... -6.948
    buses_y                                (buses_i) float64 41.25 ... 54.65
Attributes:
    network_name:           DestinE Pilot with VRE profiles
    network_pypsa_version:  0.23.0
    network_srid:           4326
    meta:                   {}